In [1]:
import csv
import random
import string
import names
import datetime

# Define generator functions

In [2]:
def gen_datetime(min_year=1900, max_year=datetime.datetime.now().year, mode = 'date'):
    if(mode == 'date'):
        start = datetime.date(min_year, 1, 1)
    elif(mode == 'datetime'):
        start = datetime.datetime(min_year, 1, 1, 00, 00, 00)
    years = max_year - min_year + 1
    end = start + datetime.timedelta(days=365 * years)
    if(mode == 'date'):
        return str(start + (end - start) * random.random())
    elif(mode == 'datetime'):
        return str(start + (end - start) * random.random())[:-7]

In [3]:
def gen_customers(num):
    result = []
    for i in range(1,num+1):
        result.append([                   
            names.get_first_name(), 
            names.get_last_name(),  
            str(gen_datetime(1900, 2000)),
            random.choice(string.ascii_letters) + random.choice(string.ascii_letters) + str(random.randint(100000, 999999)),
            random.choice(["ID", "Passport"]),
            "Authority" + str(random.randint(1,11))
            ])
    return result

In [4]:
def gen_phones(customer_list):
    result = []
    for i in range(1, len(customer_list)+1):
        for j in range(1, random.randint(2,3)):
            result.append([i, random.randint(100000000000,999999999999)])
    return result

In [5]:
def gen_emails(customer_list):
    result = []
    for i in range(1, len(customer_list)+1):
        for j in range(1, random.randint(2,3)):
            result.append([
                i,
                ''.join(random.choices(string.ascii_lowercase + string.digits, k = random.randint(4,10))) 
                + "@"
                + ''.join(random.choices(string.ascii_lowercase + string.digits, k = random.randint(4,8)))
                + ".com"
                ])
    return result

In [6]:
def gen_rooms(num):
    result = []
    for i in range(1, num+1):
        result.append([random.randint(1,3), 'Room ' + str(i), 'Room on floor ' + str(i//(num//5))])
    return result   

In [7]:
def gen_special_spaces(bars, restaurants, work, gyms, saunas, hair):
    result = []
    for (num, t) in [(bars, 'Bar'), (restaurants, 'Restaurant'), 
                     (work, 'Work space'), (gyms, 'Gym'), (saunas, 'Sauna'), (hair, 'Hairdresser')]:
        for i in range(1, num+1):
            result.append([0, t + ' '+ str(i), t + ' on floor 0'])
    return result

# Generate data

In [8]:
customers = gen_customers(100)
with open('csv/Customer.csv', 'w', newline = '') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['name', 'surname', 'date_of_birth', 'id_document_number', 'id_document_type', 
                    'id_document_authority'])
    for c in customers:
        writer.writerow(c)

In [9]:
phones = gen_phones(customers)
with open('csv/instPhone.csv', 'w', newline = '') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['nfc_id', 'phone'])
    for p in phones:
        writer.writerow(p)

In [10]:
emails = gen_emails(customers)
with open('csv/instEmail.csv', 'w', newline = '') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['nfc_id', 'email'])
    for e in emails:
        writer.writerow(e)

In [11]:
services = [['service_description', 'needs_registration'], ['Drinks at the bar', 0], 
            ['Food and drinks at the restaurant', 0], ['Meeting room', 1], ['Gym', 1],
            ['Sauna', 1], ['Hairdressing', 0], ['Room stay', 1] ]
with open ('csv/Service.csv', 'w', newline = '') as csvfile:
    writer = csv.writer(csvfile)
    for s in services:
        writer.writerow(s)

In [12]:
bars, restaurants, work, gyms, saunas, hair, rooms = 6, 4, 10, 4, 10, 1, 400

room_spaces = gen_rooms(rooms)
special_spaces = gen_special_spaces(bars, restaurants, work, gyms, saunas, hair)
spaces = special_spaces + room_spaces
with open('csv/Space.csv', 'w', newline = '') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['number_of_beds',  'name', 'description_location' ])
    for s in spaces:
        writer.writerow(s)

offers = []
i = 1
j = 1
for num in [bars, restaurants, work, gyms, saunas, hair, rooms]:
    for k in range(1, num+1):
        offers.append([i, j])
        i += 1
    j += 1

with open('csv/Offers.csv', 'w', newline = '') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['space_id', 'service_id'])
    for o in offers:
        writer.writerow(o)  

## Ιδέα για να φτιάξουμε εγγραφές, προσβάσεις, επισκέψεις, χρεώσεις.

Εγγραφές:
Για κάθε πελάτη διάλεξε μία ημερομηνία στους τελευταίους 2 μήνες, εκείνη τη μέρα **εγγράφεται** σε κάποιες υπηρεσίες και αποκτά **πρόσβαση** στους χώρους τους, επιπλέον εγγράφεται σίγουρα στο room stay και αποκτά πρόσβαση σε ένα τυχαίο δωμάτιο.

Επισκέψεις:
Από τη μέρα των εγγραφών του και για κάθε μέρα μέχρι σήμερα, ο κάθε πελάτης κάνει κάποιες ( τυχαίες ή σταθερες) **επισκέψεις** σε χώρους στους οποίους έχει πρόσβαση. σε κάθε επίσκεψη κάνει και μία **χρέωση** τυχαίου κόστους.

registers = []
visits = []
transactions = []

empty_rooms = [i[0] for i in offers if i[1] == 7]

for i in range(0,len(customers)):
    
    # Decide the services available to the customer
    registered_services = [7]
    temp = [3,4,5]
    random.shuffle(temp)
    for j in range(0, random.randint(1,3)):
        registered_services.append(temp[j])
    
    # Generate a registration datetime
    mydatetime  = datetime.datetime.now() - datetime.timedelta(days = random.randint(40,60), 
                                                                          hours = random.randint(0, 24),
                                                                          minutes = random.randint(0, 60),
                                                                          seconds = random.randint(0, 60))
    
    days_since_registration =  (datetime.datetime.now() - mydatetime).days
    
    # Registration generation for this customer is finished, append it to the registration list 
    registers.append((registered_services, str(mydatetime)[:-7]))

    # All services available to the customer
    available_services =  [1,2,6] + registered_services
    
    # The areas accessible by the customer
    a = random.choice(empty_rooms) # Pick their room
    empty_rooms.remove(a) # It is no longer empty
    available_spaces = [i[0] for i in offers if i[1] in available_services and i[1] != 7] + [a]
    
    # Create customer visits and transactions
    for j in range(1, 4*days_since_registration):
        visited_space = random.choice(available_spaces)
        arrival_datetime = mydatetime + datetime.timedelta(minutes = random.randint(1,10), 
                                                             seconds = random.randint(0,60))
        exit_datetime = arrival_datetime + datetime.timedelta(hours = random.randint(2,4), 
                                      minutes = random.randint(0,60),
                                      seconds = random.randint(0,60))
        visits.append([i+1, visited_space, str(arrival_datetime)[:-7], str(exit_datetime)[:-7]])
        if offers[visited_space-1][1] != 7:
            transactions.append([i+1, offers[visited_space-1][1], 
                                 str(exit_datetime + datetime.timedelta(minutes = random.randint(10,20), 
                                                                    seconds = random.randint(0,60)))[:-7],
                                'Used the service', round(random.uniform(1,100),3)])
        mydatetime = exit_datetime
            
# Create the Registers entries
with open('csv/Registers.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['nfc_id', 'service_id', 'registration_datetime'])
    for i in range(0, len(registers)):
        for r in registers[i][0]:
            writer.writerow([i+1, r, registers[i][1]])

with open('csv/Visits.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['nfc_id', 'space_id', 'arrival_datetime', 'exit_datetime'])
    for v in visits:
        writer.writerow(v)
        
with open('csv/Service_charge.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['nfc_id', 'service_id', 'service_charge_datetime', 'description', 'amount'])
    for t in transactions:
        writer.writerow(t)